# Web-Scraping using Regular Expressions

###  Steps 
1. Import packages
2. Get the text data from webpages from years 2015 to 2023 for www.apartments.com using BeautifulSoup
3. Remove HTML tags and clean the text data
4. Use regular expressions to find address, bed range, price range and contact number for each rental property from years 2015 to 2023
5. Convert extracted information to a Data Frame and save the Data Frame

## Step 1: Import Packages

In [8]:
import requests
import time
import os
import pandas as pd
import numpy as np
import random
import re
import regex
from time import sleep
from os import path
from pandas import DataFrame
from bs4 import BeautifulSoup
from random import randint

## Step 2 and 3 : Get the text data from webpages using BeautifulSoup and Remove HTML tags and clean the text data 

In [9]:
def get_contents(soup, content_text):
  try:
    parents_blacklist=['[document]','html','head',
                       'style','script','body',
                       'section','tr',
                       'td','label','ul','header',
                       'aside']
    content=''
    text=soup.find_all(text=True)

    
    for t in text:
        if t.parent.name not in parents_blacklist and len(t) > 5:
            content=content+t+' '
    content_text.append(content)
  except Exception:
    content_text.append('')
    pass

## Step 4: Use regular expressions to find address, bed range, price range and contact number 

In [10]:
def localized(text):
    contacts = r'\(?[0-9]{3}\)?\/?\.?-?\s?[0-9]{3}-?\.?-?\s?[0-9]{4}\b'
    address = r'(?<!\S)\b([1-9][0-9]{,4}\s*-\s*)?[1-9][0-9]{,4} ([A-Za-z0-9&]*,?\s*){,5} (Chicago|CHICAGO),?[\w\s,]{,3}\s*[0-9]{,6}?\b'
    bed_range = r'((?<!\S)\b(Studio|[1-9])\s*-\s*[1-9]\s*(Bed|Br|Beds|Bedrooms)\b)'
    prices = r'\$[1-9]?[0-9]{,3},?[0-9]{,3}\s*-\s*\$?[1-9]?[0-9]{,3},?[0-9]{,3}|\$[1-9]?[0-9]{,3},?[0-9]{,3}$|Call for Rent\b'
    output_final = []
    if re.search(address, text).groups() is not None:
        for i in re.finditer(address, text):
            property_info = {'Address': None, 'Price': None, 'Beds': None, 'Contact' : None}
            count = 0
            indexlocation= i.span()
            startindex= i.start()
            endindex= i.end()
            if indexlocation[0] >= 0 and indexlocation[1] + 100 < len(text):
                out = text[indexlocation[0]:indexlocation[1]+250] 
                address_words = (re.search(address, out).group()).split()
                if re.search(address, out):
                    property_info['Address'] = " ".join(address_words[-6:])
                    count += 1
                if re.search(prices, out):
                    property_info['Price'] = re.search(prices, out).group()
                    count += 1
                if re.search(bed_range,out):
                    property_info['Beds'] = re.search(bed_range, out).group()
                    count += 1
                if re.search(contacts,out):
                    property_info['Contact'] = re.search(contacts, out).group()
                    count += 1
                if count > 2:
                    output_final.append(property_info)
        merged = pd.DataFrame(output_final)
        print(merged)
        return merged
    else:
        print("Pattern Doesn't Match")
        return None
        

## Step 5: Convert extracted information to a Data Frame and save the Data Frame

In [12]:
links_df = pd.read_csv("results/links.csv")
for i, row in links_df.iterrows():
    webpage = row['Links']
    stamp = webpage[28:42]
    print(webpage, stamp)
    page=requests.get(webpage)
    soup=BeautifulSoup(page.text,'html.parser')
    content_text = []
    get_contents(soup, content_text)
    output_df = localized(content_text[0])
    file_name = "results_4/" + stamp + ".csv"
    output_df.to_csv(file_name)
    print("Saved to ", file_name)

https://web.archive.org/web/20150217144133/https://www.apartments.com/chicago-il/ 20150217144133
                              Address             Price           Beds  \
0        360 E South Water St Chicago   $1,590 - $3,953  Studio - 2 Br   
1             71 W Hubbard St Chicago   $1,829 - $6,947  Studio - 3 Br   
2             345 E Wacker Dr Chicago   $1,568 - $4,530  Studio - 3 Br   
3   5020-5050 S Lake Shore Dr Chicago   $1,113 - $3,274  Studio - 3 Br   
4        400 E South Water St Chicago   $1,588 - $3,802  Studio - 2 Br   
5               500 E 33rd St Chicago     $725 - $1,866  Studio - 3 Br   
6           225 N Columbus Dr Chicago   $1,612 - $4,605  Studio - 2 Br   
7         1130 S Michigan Ave Chicago   $1,190 - $3,896  Studio - 2 Br   
8           200 E Illinois St Chicago   $2,226 - $9,700  Studio - 4 Br   
9        1350 N Lake Shore Dr Chicago   $1,191 - $3,032  Studio - 2 Br   
10              750 N Rush St Chicago   $1,794 - $6,712         1-3 Br   
11        850 N

                           Address             Price           Beds  \
0          345 E Wacker Dr Chicago   $1,611 - $4,121  Studio - 3 Br   
1         805 N LaSalle Dr Chicago   $1,600 - $5,100  Studio - 2 Br   
2        200 E Illinois St Chicago  $1,898 - $12,878  Studio - 4 Br   
3          111 W Wacker Dr Chicago   $1,816 - $7,584  Studio - 3 Br   
4          1 W Superior St Chicago   $1,495 - $5,359  Studio - 2 Br   
5        37 W Van Buren St Chicago   $1,339 - $3,525  Studio - 4 Br   
6           900 S Clark St Chicago   $1,339 - $3,525  Studio - 3 Br   
7           850 S Clark St Chicago   $1,561 - $3,578  Studio - 2 Br   
8         400 N McClurg Ct Chicago   $1,438 - $2,728  Studio - 2 Br   
9      850 N Lake Shore Dr Chicago     Call for Rent  Studio - 3 Br   
10    400 E South Water St Chicago   $1,558 - $3,301  Studio - 2 Br   
11    340 E North Water St Chicago  $1,969 - $13,500  Studio - 3 Br   
12     1130 S Michigan Ave Chicago   $1,284 - $3,049  Studio - 2 Br   
13    

                          Address             Price           Beds  \
0           11 S Green St Chicago   $1,675 - $3,335  Studio - 2 Br   
1           500 E 33rd St Chicago     $837 - $2,341  Studio - 3 Br   
2       225 N Columbus Dr Chicago   $1,634 - $5,050  Studio - 2 Br   
3         345 E Wacker Dr Chicago   $1,636 - $4,688  Studio - 3 Br   
4   5140 S Hyde Park Blvd Chicago   $1,398 - $5,781         1-3 Br   
5          910 W Huron St Chicago   $2,460 - $3,630  Studio - 4 Br   
6       37 W Van Buren St Chicago   $1,899 - $1,999  Studio - 4 Br   
7     1130 S Michigan Ave Chicago   $1,284 - $3,460  Studio - 2 Br   
8        400 N McClurg Ct Chicago   $2,128 - $2,926  Studio - 2 Br   
9    1350 N Lake Shore Dr Chicago   $1,272 - $2,994  Studio - 2 Br   
10  3402-3453 N Elaine Pl Chicago   $1,672 - $3,488         1-3 Br   
11   340 E North Water St Chicago  $2,174 - $10,185  Studio - 3 Br   
12       555 W Madison St Chicago   $1,315 - $4,000  Studio - 2 Br   
13      200 E Illino

                           Address             Price           Beds  \
0    5140 S Hyde Park Blvd Chicago   $1,398 - $5,781         1-3 Br   
1          345 E Wacker Dr Chicago   $1,633 - $4,688  Studio - 3 Br   
2     123 N Des Plaines St Chicago   $2,389 - $5,377  Studio - 3 Br   
3            11 S Green St Chicago   $1,675 - $3,335  Studio - 2 Br   
4         555 W Madison St Chicago   $1,286 - $4,000  Studio - 2 Br   
5     360 E South Water St Chicago   $1,715 - $3,884  Studio - 2 Br   
6          161 W Kinzie St Chicago   $1,840 - $6,337  Studio - 2 Br   
7         400 N McClurg Ct Chicago   $1,845 - $2,926  Studio - 2 Br   
8      500 N Lake Shore Dr Chicago   $2,235 - $3,825  Studio - 2 Br   
9         805 N LaSalle Dr Chicago   $1,600 - $5,100  Studio - 2 Br   
10   401-465 E Illinois St Chicago   $1,775 - $4,445  Studio - 2 Br   
11       225 N Columbus Dr Chicago   $1,634 - $5,117  Studio - 2 Br   
12          910 W Huron St Chicago   $1,950 - $3,410  Studio - 4 Br   
13  11

                           Address             Price           Beds  \
0          161 W Kinzie St Chicago   $1,880 - $6,337  Studio - 2 Br   
1         555 W Madison St Chicago   $1,350 - $3,985  Studio - 2 Br   
2        37 W Van Buren St Chicago   $1,279 - $2,994  Studio - 4 Br   
3     1350 N Lake Shore Dr Chicago   $1,279 - $2,994  Studio - 2 Br   
4      500 N Lake Shore Dr Chicago   $1,755 - $3,945  Studio - 2 Br   
5   1164-1212 W Madison St Chicago   $1,875 - $3,250  Studio - 2 Br   
6        200 E Illinois St Chicago  $2,191 - $12,878  Studio - 4 Br   
7          71 W Hubbard St Chicago   $1,795 - $6,495  Studio - 3 Br   
8      1130 S Michigan Ave Chicago   $1,374 - $3,443  Studio - 2 Br   
9            11 S Green St Chicago   $1,584 - $3,425  Studio - 2 Br   
10           500 E 33rd St Chicago     $879 - $2,000  Studio - 3 Br   
11   5140 S Hyde Park Blvd Chicago   $1,378 - $5,781         1-3 Br   
12    400 E South Water St Chicago   $1,660 - $4,130  Studio - 2 Br   
13    

...

                                Address            Price            Beds  \
0          10 Apartments in Chicago 727   $1,735 - 7,585  Studio - 3 Bed   
1         727 W Madison St, Chicago, IL   $1,735 - 7,585  Studio - 3 Bed   
2            465 N Park Dr, Chicago, IL  $1,710 - 13,000  Studio - 3 Bed   
3           720 S Wells St, Chicago, IL   $1,755 - 7,589  Studio - 3 Bed   
4      2300 S Michigan Ave, Chicago, IL   $1,520 - 2,705         1-2 Bed   
5        311 W Illinois St, Chicago, IL   $1,875 - 5,860  Studio - 3 Bed   
6         555 W Madison St, Chicago, IL   $1,450 - 3,265  Studio - 2 Bed   
7          175 N Harbor Dr, Chicago, IL   $1,718 - 5,281  Studio - 3 Bed   
8          1350 N Wells St, Chicago, IL   $1,570 - 4,030  Studio - 2 Bed   
9          4334 N Hazel St, Chicago, IL   $1,460 - 2,345  Studio - 2 Bed   
10      4-22 N Aberdeen St, Chicago, IL   $2,795 - 3,920  Studio - 3 Bed   
11       320 W Illinois St, Chicago, IL   $1,435 - 3,500  Studio - 2 Bed   
12   Martin 

                                Address            Price            Beds  \
0              10 Apartments in Chicago   $1,695 - 4,500  Studio - 2 Bed   
1        1457 N Halsted St, Chicago, IL   $1,695 - 4,500  Studio - 2 Bed   
2          633 W North Ave, Chicago, IL   $1,670 - 7,575  Studio - 3 Bed   
3        811 W Agatite Ave, Chicago, IL   $1,530 - 5,880  Studio - 3 Bed   
4          728 N Morgan St, Chicago, IL   $1,250 - 5,942  Studio - 3 Bed   
5         930 W Altgeld St, Chicago, IL  $1,445 - 14,020  Studio - 3 Bed   
6        NE NEast River Rd, Chicago, IL     $835 - 1,945  Studio - 4 Bed   
7          2 W Delaware Pl, Chicago, IL   $3,660 - 9,979         1-5 Bed   
8      1407 S Michigan Ave, Chicago, IL   $1,475 - 5,575  Studio - 3 Bed   
9           W Van Buren St, Chicago, IL   $1,378 - 2,950  Studio - 2 Bed   
10         29 S LaSalle St, Chicago, IL   $1,660 - 7,000  Studio - 2 Bed   
11          W Van Buren St, Chicago, IL   $1,710 - 3,265  Studio - 2 Bed   
12   1050-10

                                Address            Price            Beds  \
0              10 Apartments in Chicago  $2,050 - 14,435  Studio - 3 Bed   
1         930 W Altgeld St, Chicago, IL  $2,050 - 14,435  Studio - 3 Bed   
2        NE NEast River Rd, Chicago, IL   $1,065 - 2,590  Studio - 4 Bed   
3          2 W Delaware Pl, Chicago, IL  $3,548 - 10,551         1-5 Bed   
4        1457 N Halsted St, Chicago, IL   $2,474 - 3,895  Studio - 2 Bed   
5       808 S Michigan Ave, Chicago, IL  $1,528 - 39,671  Studio - 4 Bed   
6               255-3461 One Chicago 14  $2,259 - 10,680  Studio - 3 Bed   
7           S East End Ave, Chicago, IL   $1,475 - 3,632  Studio - 4 Bed   
8           W Van Buren St, Chicago, IL   $2,027 - 3,860         1-2 Bed   
9      1407 S Michigan Ave, Chicago, IL   $1,745 - 3,800  Studio - 2 Bed   
10         N Lake Shore Dr, Chicago, IL   $1,528 - 8,340  Studio - 2 Bed   
11  1164-1212 W Madison St, Chicago, IL   $1,922 - 4,546  Studio - 2 Bed   
12         N

                                    Address            Price             Beds  \
0                  10 Apartments in Chicago   $2,246 - 4,199  Studio - 2 Beds   
1            1457 N Halsted St, Chicago, IL   $2,246 - 4,199  Studio - 2 Beds   
2             930 W Altgeld St, Chicago, IL  $2,150 - 14,380  Studio - 3 Beds   
3            NE NEast River Rd, Chicago, IL   $1,195 - 3,190  Studio - 4 Beds   
4               S East End Ave, Chicago, IL   $1,575 - 4,789  Studio - 4 Beds   
5           5029 N Kenmore Ave, Chicago, IL   $2,575 - 6,895         1-3 Beds   
6                 8 E Huron St, Chicago, IL   $2,575 - 6,895         1-3 Beds   
7               1616 E 56th St, Chicago, IL   $2,650 - 7,589         1-3 Beds   
8                 205 S Peoria, Chicago, IL   $2,120 - 4,818  Studio - 2 Beds   
9       1164-1212 W Madison St, Chicago, IL   $2,846 - 5,610  Studio - 2 Beds   
10              730 W Couch Pl, Chicago, IL   $2,204 - 9,367  Studio - 3 Beds   
11             1 W Superior 

                               Address            Price             Beds  \
0             10 Apartments in Chicago   $2,309 - 5,936  Studio - 2 Beds   
1       1457 N Halsted St, Chicago, IL   $2,309 - 5,936  Studio - 2 Beds   
2           234 W Polk St, Chicago, IL   $2,170 - 5,365  Studio - 3 Beds   
3      5630 N Sheridan Rd, Chicago, IL   $1,130 - 1,550   Studio - 1 Bed   
4        930 W Altgeld St, Chicago, IL  $2,170 - 14,700  Studio - 3 Beds   
5      5252 S Cornell Ave, Chicago, IL   $2,450 - 9,853  Studio - 3 Beds   
6     1950 N Campbell Ave, Chicago, IL   $2,400 - 3,900         1-2 Beds   
7       121 W Chestnut St, Chicago, IL  $2,241 - 11,979  Studio - 3 Beds   
8        E North Water St, Chicago, IL  $2,234 - 12,074  Studio - 3 Beds   
9    2552 N Milwaukee Ave, Chicago, IL   $2,040 - 5,000  Studio - 2 Beds   
10    Milieu 205 S Peoria, Chicago, IL   $2,217 - 4,826  Studio - 2 Beds   
11      1210 N State Pkwy, Chicago, IL   $2,990 - 6,423  Studio - 2 Beds   
12          